In [5]:
!pip install -r requirements.txt
!pip freeze
!pip install --upgrade bokeh

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
appnope==0.1.0
attrs==19.3.0
backcall==0.1.0
bleach==3.1.1
bokeh==1.4.0
decorator==4.4.1
defusedxml==0.6.0
entrypoints==0.3
importlib-metadata==1.5.0
ipykernel==5.1.4
ipython==7.12.0
ipython-genutils==0.2.0
jedi==0.16.0
Jinja2==2.11.1
jsonschema==3.2.0
jupyter-client==6.0.0
jupyter-core==4.6.3
lxml==4.5.0
MarkupSafe==1.1.1
mistune==0.8.4
nbconvert==5.6.1
nbformat==5.0.4
notebook==6.0.3
numpy==1.18.1
packaging==20.1
pandocfilters==1.4.2
parso==0.6.1
pexpect==4.8.0
pickleshare==0.7.5
Pillow==7.0.0
prometheus-client==0.7.1
prompt-toolkit==3.0.3
ptyprocess==0.6.0
Pygments==2.5.2
pyparsing==2.4.6
pyrsistent==0.15.7
python-dateutil==2.8.1
PyYAML==5.3
pyzmq==19.0.0
regex==2020.2.20
segtok==1.5.7
Send2Trash==1.5.0
six==1.14.0
terminado==0.8.3
testpath==0.4.4
tornado==6.0.3
traitlets==4.3.3
virtualenv==16.7.2
wcwidth==0.1.8
webencodings==0.5.1
zipp==3.0.0
Requirement already up-to-date: bokeh in /us

In [2]:
from lxml import etree as ET
from bokeh.io import show, output_file
from bokeh.plotting import figure
import regex as re

In [21]:
with open("textv2G1.xml") as f:
    xml = ET.parse(f)
    
counts = []
identifiers = []

manuscrit = "G1"
# Je récupère les @types et je les mets dans une liste de valeur unique (set) puis je les trie (sorted)
cats = sorted(list(set([str(x) for x in xml.xpath("//@type")])))

# Passage pour compter les mots
_punc = re.compile("([^<>/\w\s])")
_spaces = re.compile("([\s\u00a0]+)")

def count_words(text):
    text = _spaces.sub(" ", _punc.sub(" ", text))
    return len([x for x in text.split() if x ]), text
# Fin des définitions d'outils pour compter les mots

# Pour chaque chapitre et chaque "pas" de chapitre (0, chapitre1), (1, chapitre2)
for chap_no, chap in enumerate(xml.xpath("./div")):
    chap_no += 1
    for sect_no, section in enumerate(chap.xpath("./div")):
        sect_no += 1
        for p_no, p in enumerate(section.xpath("./p")):
            p_no += 1
            # Création d'une chaine identifiant nb_chap.nb_sect.nb_p
            identifier = "{}.{}.{}".format(chap_no, sect_no, p_no)
            # J'ajoute aussi un numero sequentiel de paragraph: les paragraphes sont numérotés de 1 à la fin
            identifiers.append(len(identifiers) + 1)
            # Création du dictionnaire où il y aura les stats1
            stats = {
                "identifier": identifier,
                "neutral": 0,
                "edition": 0,
                manuscrit: 0,
                **{cat: 0 for cat in cats}
            }
            
            # Récupération du texte non-spécifique
            for text_node in p.xpath("./text()"):
                cnt, txt = count_words(str(text_node).strip())
                if cnt:
                    stats["neutral"] += cnt
            
            # Récupération du texte spécifique à l'Édition (et donc en contradiction avec autre manuscrit)
            for text_node in p.xpath(".//Edition/text()"):
                cnt, txt = count_words(str(text_node).strip())
                if cnt:
                    stats["edition"] += cnt
            
            # Pour calcul total, addition du spécifique à l'édition et du général
            stats["edition_real"] = stats["edition"] + stats["neutral"]
            
            # Et maintenant, par catégorie de type
            for manuscrit_var in p.xpath(".//{}/text()".format(manuscrit)):
                cnt, txt = count_words(str(manuscrit_var).strip())
                if cnt:
                    # Je rajoute aussi en fonction de @type
                    stats[manuscrit_var.getparent().attrib["type"]] += cnt
                    # On ajoute aussi au général
                    stats[manuscrit] += cnt
            
            stats["diff"] = stats["edition"] - stats[manuscrit]
            counts.append(stats)


In [22]:
import csv

# On exporte le CSV
header = list(counts[0].keys())
with open("stats.csv", "w") as f:
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    writer.writerows(counts)

In [23]:
# Optionnel: recharger depuis CSV

cats = ['ajout', 'inv.', 'om.', 'plur.', 'red.', 'semantique', 'sing.', 'temps']
import csv
counts = []
identifiers = []
def to_int(string):
    if string.isnumeric():
        return int(string)
    return string
with open("./statsG1.csv") as f:
    reader = csv.DictReader(f)
    for line in reader:
        if line["neutral"].isnumeric() and line["identifier"]:
            counts.append({key: to_int(val) for key, val in line.items()})
            identifiers.append(line["identifier"])
print(counts)

[{'identifier': '1.1.1', 'neutral': 162, 'edition': 101, 'G1': 86, 'ajout': 17, 'inv.': 3, 'om.': 5, 'plur.': 0, 'red.': 3, 'semantique': 57, 'sing.': 0, 'temps': 1, 'text': 0, 'edition_real': 263, 'diff': 15, '': ''}, {'identifier': '2.1.1', 'neutral': 330, 'edition': 158, 'G1': 129, 'ajout': 11, 'inv.': 11, 'om.': 18, 'plur.': 0, 'red.': 4, 'semantique': 83, 'sing.': 2, 'temps': 0, 'text': 0, 'edition_real': 488, 'diff': 29, '': ''}, {'identifier': '3.1.1', 'neutral': 197, 'edition': 82, 'G1': 91, 'ajout': 0, 'inv.': 3, 'om.': 6, 'plur.': 0, 'red.': 0, 'semantique': 82, 'sing.': 0, 'temps': 0, 'text': 0, 'edition_real': 279, 'diff': '-9', '': ''}, {'identifier': '4.1.1', 'neutral': 215, 'edition': 38, 'G1': 43, 'ajout': 6, 'inv.': 2, 'om.': 4, 'plur.': 0, 'red.': 0, 'semantique': 29, 'sing.': 0, 'temps': 2, 'text': 0, 'edition_real': 253, 'diff': '-5', '': ''}, {'identifier': '5.1.1', 'neutral': 267, 'edition': 58, 'G1': 66, 'ajout': 0, 'inv.': 5, 'om.': 11, 'plur.': 0, 'red.': 0, 's

In [24]:
# Revert the dict !


from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.palettes import all_palettes

output_file("bar_stacked.html")


# Remap data
data = {"identifier": [], **{cat: [] for cat in cats}}
psg_size = [psg["edition_real"] for psg in counts]
for psg_id, element in enumerate(counts):
    for key in data:
        if key != "identifier":
            data[key].append(element[key] / psg_size[psg_id])
        else:
            data[key].append(element[key]) 
        
data["identifier"] = [str(x) for x in identifiers]

# Trie les catégories dans l'ordre décroissant de décompte total
cats = sorted(cats, key=lambda x: -sum(data[x]))

# Plot
p = figure(x_range=data["identifier"], plot_width=1280, plot_height=800, title="Type of modification per passage",
           toolbar_location=None, tools="hover", tooltips="Var[type='$name'][p=@identifier]: @$name")
p.vbar_stack(cats, x='identifier', width=0.9, source=data, legend_label=cats, 
             color=all_palettes["Category20"][len(cats)])

#p.line(data["identifier"], psg_size, color='#000000', legend_label='Size of the edition passage')

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_left"
p.legend.orientation = "horizontal"

show(p)